## Preprocessing

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [11]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
clean_df = application_df.drop(columns = ["EIN","NAME","STATUS","SPECIAL_CONSIDERATIONS"])
clean_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [12]:
# Determine the number of unique values in each column.
print(clean_df.nunique())

APPLICATION_TYPE      17
AFFILIATION            6
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           4
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64


In [13]:
# Look at APPLICATION_TYPE value counts for binning
app_count = clean_df["APPLICATION_TYPE"].value_counts()
app_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [14]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace =list(app_count[app_count < 500].index)


# Replace in dataframe
for app in application_types_to_replace:
        clean_df['APPLICATION_TYPE'] = clean_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
clean_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [15]:
# Look at CLASSIFICATION value counts for binning
classification_count = clean_df["CLASSIFICATION"].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [16]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classification_count1 = classification_count.loc[classification_count >1]
classification_count1.head(10)

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
Name: CLASSIFICATION, dtype: int64

In [17]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(classification_count[classification_count <1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    clean_df['CLASSIFICATION'] = clean_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
clean_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
clean_df_1 = pd.get_dummies(clean_df)
clean_df_1.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [19]:
# Split our preprocessed data into our features and target arrays
X = clean_df_1.drop("IS_SUCCESSFUL", axis = 1)
y = clean_df_1["IS_SUCCESSFUL"]

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=58)

In [20]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 6


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                410       
                                                                 
 dense_1 (Dense)             (None, 8)                 88        
                                                                 
 dense_2 (Dense)             (None, 6)                 54        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 559
Trainable params: 559
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 100
                   )

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6312 - accuracy: 0.6594
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5686 - accuracy: 0.7291
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5613 - accuracy: 0.7298
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5589 - accuracy: 0.7299
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5577 - accuracy: 0.7301
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7308
Epoch 7/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7322
Epoch 8/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5549 - accuracy: 0.7311
Epoch 9/100
804/804 [==============================] - 3s 4ms/step - loss: 0.5537 - accuracy: 0.7320
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accura

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5484 - accuracy: 0.7314 - 833ms/epoch - 3ms/step
Loss: 0.5484125018119812, Accuracy: 0.7314285635948181


## Third Attempt


In [25]:
# Third Attempt to optimize the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 30
                   )


Epoch 1/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5432 - accuracy: 0.7357
Epoch 2/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5435 - accuracy: 0.7358
Epoch 3/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5433 - accuracy: 0.7354
Epoch 4/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5436 - accuracy: 0.7353
Epoch 5/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5433 - accuracy: 0.7351
Epoch 6/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5433 - accuracy: 0.7363
Epoch 7/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5434 - accuracy: 0.7346
Epoch 8/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5433 - accuracy: 0.7362
Epoch 9/30
804/804 [==============================] - 2s 3ms/step - loss: 0.5433 - accuracy: 0.7355
Epoch 10/30
804/804 [==============================] - 2s 2ms/step - loss: 0.5433 - accuracy: 0.7353

In [26]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5493 - accuracy: 0.7297 - 379ms/epoch - 1ms/step
Loss: 0.549319326877594, Accuracy: 0.72967928647995


## Fourth Attempt


In [27]:
# Fourth Attempt to optimize the model
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  10
hidden_nodes_layer2 = 8
hidden_nodes_layer3 = 6
hidden_nodes_layer4 = 2


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="sigmoid"))

# Third hidden layer

nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="sigmoid"))

#Fourth Hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="sigmoid"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                410       
                                                                 
 dense_5 (Dense)             (None, 8)                 88        
                                                                 
 dense_6 (Dense)             (None, 6)                 54        
                                                                 
 dense_7 (Dense)             (None, 2)                 14        
                                                                 
 dense_8 (Dense)             (None, 1)                 3         
                                                                 
Total params: 569
Trainable params: 569
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs= 100
                   )

Epoch 1/100
804/804 [==============================] - 3s 2ms/step - loss: 0.6592 - accuracy: 0.6187
Epoch 2/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5894 - accuracy: 0.7298
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5769 - accuracy: 0.7315
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5744 - accuracy: 0.7312
Epoch 5/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5731 - accuracy: 0.7322
Epoch 6/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5724 - accuracy: 0.7324
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5714 - accuracy: 0.7320
Epoch 8/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5706 - accuracy: 0.7318
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5698 - accuracy: 0.7323
Epoch 10/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5691 - accura

In [30]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5509 - accuracy: 0.7310 - 491ms/epoch - 2ms/step
Loss: 0.5509160161018372, Accuracy: 0.7309620976448059


In [31]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimized.h5", save_format = "h5")